# 1. Long sequence(longseq)
 - 이전에는 RNN으로 짧은 문장이나 단어를 학습했다.
 - RNN으로 진짜 유의미한 결과를 얻고 싶다면 긴 문장인 longseq에 도전하자!
 - 긴 문장을 특정 균일 사이즈로 잘라서 이용하자.

# 2. 긴 문장을 조각 데이터로 만들기
### (1) 긴 문장을 조각 데이터로 만드는 법
 - 긴 문장 위에 10칸짜리 윈도우를 두고, 오른쪽으로 한칸씩 옮겨가며 데이터셋을 만든다.

<br>

### (2) long sentence

In [1]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
sequence_length = len(sentence)

<br>

### (3) sequence dataset

```python
   x_data    ->    y_data
"if you wan" -> "f you want"
"f you want" -> " you want "
" you want " -> "you want t"
"you want t" -> "ou want to"
"ou want to" -> "u want to "
```

# 3. 긴 문장을 조각 데이터로 만들기 코드

In [2]:
import torch

# 데이터 담을 리스트
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length): # 문장의 인덱스 0부터, '전체 문장 길이 - 윈도우 길이'만큼 for문을 돈다.
  x_str = sentence[i : i + sequence_length] # x_str = i 부터 'i + 윈도우 길이'
  y_str = sentence[i+1 : i + sequence_length + 1] # y_str = i+1 부터 'i + 1 + 윈도우 길이'
  print(i, x_str, '->', y_str)
  x_data.append([char_dic[c] for c in x_str]) # 문장을 인덱스로 바꿔준다.
  y_data.append([char_dic[c] for c in y_str])
  
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # One_hot encoding

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

# 4. RNN 레이어 쌓기 실습하기
 - RNN 레이어를 2개 쌓고(stacking), FC layer를 추가 해 보자.

In [3]:
# RNN 모델 만들어 놓기
class Net(torch.nn.Module): # PyTorch 모듈을 상속받아서 lass 정의
  def __init__(self, input_dim, hidden_dim, layers): # 하위 모듈 이용 정의
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True) # N개의 레이어를 갖는 RNN
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True) # FC Layer

  def forward(self, x):
    x, _status = self.rnn(x) # RNN
    x = self.fc(x) # FC Layer
    return x

In [4]:
# 문자 집합
char_set = list(set(sentence)) # char_set은 character의 집합이다. set 함수로 sample에서 중복된 문자를 제거하고 유니크한 문자만 모아서 리스트로 만든다.
char_dic = {c: i for i, c in enumerate(char_set)} # char_dic은 특정 문자를 주면 그 인덱스를 찾아준다. enumerate를 쓰면 인덱스와 밸류를 같이 가져올 수 있다.

# 하이퍼 파라미터 설정
dic_size = len(char_dic)
hidden_size = len(char_dic)
learning_rate = 0.1

# RNN 모델 이용
net = Net(dic_size, hidden_size, 2) # 모델은 2개의 RNN 레이어와 FC Layer를 갖는다!

# Loss와 Optimizer 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), learning_rate)

In [5]:
# RNN 모델 학습 시작
for i in range(100):
  outputs = net(X) # 예측값
  loss = criterion(outputs.view(-1, dic_size), Y.view(-1)) # 예측값과 실제값으로 loss 구하기

  optimizer.zero_grad() # gradient 초기화
  loss.backward() # BP
  optimizer.step() # weight 업데이트
  
  # RNN 모델이 예측한 결과물을 사람이 알아보게 해석하기
  results = outputs.argmax(dim=2) # argmax로 prediction 확률이 가장 높은 것을 구한다.
  predict_str = ""
  for j, result in enumerate(results): # 반복문을 돌며 
    print(i, j, ''.join([char_set[t] for t in result]), loss.item())
    if j==0:
      predict_str += ''.join([char_set[t] for t in result]) # 처음에는 result 중 windows 크기만큼 가져오고
    else:
      predict_str += char_set[result[-1]] # 그 이후에는 result의 마지막 부분만 반복하여 가져온다.

RuntimeError: input must have 3 dimensions, got 1